
# **Twitter Sentiment Analysis**


##  Table of Contents:
* [Import data and modules](#first-bullet)
* [Data Cleaning](#second-bullet)
* [Exploratory Analysis](#third-bullet)
    * [Classify data into parties](#section_3_1)
* [Sentiment Classification](#fourth-bullet)
    * [Machine learning models using Tf-idf](#section_4_1)
    * [Deep Learning models using Tf (Bag of words)](#section_4_2)
    * [Comparing Models](#section_4_3)
    * [Best Model Training](#section_4_4)
    * [Fitting on elections data](#section_4_5)
* [Negative Reason Analysis](#Fifth-bullet)

In [ ]:
#Importing required libraries
import pandas as pd
import numpy as np
import cvxpy as cp

import re
import string
import html
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Markdown, display
%matplotlib inline

import io
from IPython.display import HTML
from pandas import DataFrame

#Installing required libraries
#!pip install <library_name>

try:
    from sklearn import metrics
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, auc
    from sklearn.metrics import precision_score, recall_score, f1_score
    from sklearn.feature_selection import chi2, SelectFromModel, SelectKBest
    from sklearn.feature_selection import RFE
    from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold, KFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.svm import SVC
    from sklearn.svm import LinearSVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.neural_network import MLPClassifier
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn import ensemble
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, label_binarize, MinMaxScaler
    from sklearn.decomposition import PCA

except:
    !pip install -U scikit-learn
    
    from sklearn import metrics
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, auc
    from sklearn.metrics import precision_score, recall_score, f1_score
    from sklearn.feature_selection import chi2, SelectFromModel, SelectKBest
    from sklearn.feature_selection import RFE
    from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold, KFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.svm import SVC
    from sklearn.svm import LinearSVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.neural_network import MLPClassifier
    
    from sklearn.tree import DecisionTreeClassifier
    from sklearn import ensemble
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, label_binarize, MinMaxScaler
    from sklearn.decomposition import PCA

try:
  import imblearn
  from imblearn.over_sampling import RandomOverSampler
  from imblearn.over_sampling import SMOTE
  from imblearn.under_sampling import RandomUnderSampler

except:
  !pip install imbalanced-learn
  import imblearn
  from imblearn.over_sampling import RandomOverSampler
  from imblearn.over_sampling import SMOTE
  from imblearn.under_sampling import RandomUnderSampler
  

from tqdm import tqdm

try:
    from pywaffle import Waffle
except:
    !pip install pywaffle
    from pywaffle import Waffle

try:
    from statsmodels.graphics.gofplots import qqplot
    import statsmodels.api as sm
except:
    !pip install statsmodels
    from statsmodels.graphics.gofplots import qqplot

try:
  from xgboost import XGBClassifier
except:
  !pip install xgboost
  from xgboost import XGBClassifier


try:
  from venn import venn
except:
  !pip install venn
  from venn import venn

!pip install emoji
import emoji

from IPython.display import Image

import torch
from scipy.sparse import coo_matrix
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#NLTK Library
try:
  import nltk
  from nltk.stem import WordNetLemmatizer
  from nltk.corpus import stopwords

except:
  !pip install nltk
  import nltk
  from nltk.stem import WordNetLemmatizer
  from nltk.corpus import stopwords


nltk.download("stopwords") 
nltk.download('punkt')
nltk.download('wordnet')
from emoji.unicode_codes import UNICODE_EMOJI
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))  

# Section 1 : Data Cleaning

## 1.1 Loading and exploring Data 
a) Load sentiment_analysis.csv and Canadian_elections_2021.csv 

In [ ]:
# load 'sentiment_analysis.csv' to Google Colab or 'sentiment_analysis.csv' file in the session before running#
try:
  from google.colab import files
  uploaded = files.upload()
except:
  pass

In [ ]:
# load 'Canadian_elections_2021.csv' to Google Colab or 'Canadian_elections_2021.csv' file in the session before running#
try:
  from google.colab import files
  uploaded = files.upload()
except:
  pass

In [ ]:
#load 'sentiment_analysis.csv' and 'Canadian_elections_2021.csv' files in the session before running#
#Load files and into dateframe

senti_df = pd.read_csv(('sentiment_analysis.csv'), low_memory=False)
election_df = pd.read_csv(('Canadian_elections_2021.csv'), low_memory=False)

In [ ]:
# Copy of both dataset for further use
senti_df_cp = senti_df.copy()
election_df_cp= election_df.copy()

In [ ]:
#Explore dataset
print('sentiment data column',senti_df.columns)
display('\n',senti_df.info)

print("--------------------------------------------------------------------------\n")

print('election data columns\n',election_df.columns)
display('\n',election_df.info)

In [ ]:
# Sample data
print('sentiment data\n') 
display(senti_df.head(4))

print('\nElection data')
display(election_df.head(4))

In [ ]:
#Plotting graph from the dataframe provided
sns.countplot(x = 'label', data = senti_df)
plt.title("Labels distribution")
plt.plot()

In [ ]:
#Top 15 tweeted hashtags in the elections dataset
election_df.text.str.extractall(r'(\#\w+)')[0].value_counts()[:15]

In [ ]:
#Top 15 mentions in the elections dataset
election_df.text.str.extractall(r'(\@\w+)')[0].value_counts()[:15]

##1.2 Cleaning dataset 


Observing the sentiments dataset, they contain a combination of emoji's, hyperlinks and stop words. The most important step for this assignment is to clean the data before analysis of the tweets. The clean_data function handles the following:
1. Converts emoji's to text
2. Removes hyperlinks
3. Convert all HTML text to ASCII
4. Remove all numeric only data.
5. Convert text to lowercase.
6. Remove all stop-words
7. Remove all punctuations.
8. Removes twitter handles
9. Remove all tags

The assignment required to assign the tweets to a particular party in the later tasks, '#' and '@' were excluded while removing the punctuations in first excecution. These are removed later once the tweets were assigned to particular parties before the model implementation.

Also additional stop word 'rt' is added to list of Standard stopwords imported from NLTK library 

In [ ]:
#Using the stopwords
stops = stopwords.words('english')

#Add other required stopwords  
stops.append('rt')

In [ ]:
def cleaned_data(dataset, rmove_handle_tag = False):

  #remove html tags and attributes
  dataset = re.sub('(<[^>]*>)','',dataset)

  #Convert_emoji's to text
  dataset = emoji.demojize(dataset)

  #Convert character codes to string
  dataset = html.unescape(dataset)

  #remove urls
  dataset = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',dataset, flags=re.MULTILINE)

  #lower case
  dataset = dataset.lower()

  if rmove_handle_tag==False:
    #remove punctuations apart from the @ and # which can be useful later
    dataset = dataset.translate(str.maketrans('', '', re.sub('#|@','',(string.punctuation))))
  else:
    dataset = re.sub(r'\@\w+|\#\w+','', dataset)
    dataset = dataset.translate(str.maketrans('', '', (string.punctuation)))
    
  #Remove all stop-words
  dataset = [word for word in str(dataset).split() if word not in stops]
  dataset = (" ").join(dataset)
  
  #Join all the words using ' ' before returning the tweet in the dataset
  return str(dataset)

Use Function clean_data for data cleaning of both the dataset.

In [ ]:
#Processing sentiment analysis dataset
senti_df_cp['text'] = senti_df['text'].apply(lambda x: cleaned_data(x))

#Processing canadian election dataset
election_df_cp['text'] = election_df['text'].apply(lambda x: cleaned_data(x))

In [ ]:
senti_df_cp['text']

In [ ]:
# Print sample of processed tweets
print(senti_df_cp['text'][200])
print(election_df_cp['text'][200])

In [ ]:
election_df_cp.head()

#Section 2: Exploratory Data Analysis

## **2.1 Steps for Assigning Tweets to a Party**

To identify and link tweets to a political party following reference can be helpful

1. Looking at the political parties and their leaders twitter handles e.g., @libral_party, @the jagmeetsingh can be reference and linked to the party. 
2. looking at party mentions in the tweets with hashtags e.g., #ForwardForEveryone, can be reference and linked to the party. 
3. identifying keywords realted to political parties and linkin them.


Inorder to identify handles, hashtages and keywords from the tweets, visualtization is done by plotting wordclouds.



**Analyzing Hashtags and Handles**

The below cell defnies function for extracting handles and hashtags. These are then added to a new column respectively in the dataframe. 

In [ ]:
#function to find handles
def find_handles(text_data):
  text_data = re.findall('@\w+', text_data)
  return [re.sub('@','',x) for x in text_data]

election_df_cp['handles'] = election_df_cp['text'].apply(lambda x: find_handles(x))

In [ ]:
#function to find hashtags
def find_hashtags(text_data):
  text_data = re.findall('#\w+', text_data)
  return [re.sub('#','',x) for x in text_data]

election_df_cp['hashtags'] = election_df_cp['text'].apply(lambda x: find_hashtags(x))

Wordclouds for extracted handles and hashtages stored in the dataframe

In [ ]:
#Handles in the list format are exploded and joined for to plot wordcloud.
handle_data = election_df_cp['handles'].explode().unique()

handle_data_drop = ' '.join(np.delete(handle_data, 0))

wordcloud = WordCloud(random_state=21, max_font_size=30,background_color ='lightblue').generate(handle_data_drop)

#Plotting Wordcloud for handles
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('WordCloud for the Canadian Election handles')
plt.axis('off')
plt.show()

Some of the handles that are most frequent are: 'justintrudeau', 'thejagmeetsingh', 'erinotootle', 'codyotootle', 'cafreeland', 'maxinebernier', 'peoplespca' etc. These handles can be referenced with their affliation to a party.
It will help in defining keywords for party allocation

In [ ]:
#Handles in the list format are exploded and joined for to plot wordcloud.
hashtags_data = election_df_cp['hashtags'].explode().unique()

hashtags_data_drop = ' '.join(np.delete(hashtags_data, 0))

wordcloud = WordCloud(random_state=21, max_font_size=30, background_color ='lightblue').generate(hashtags_data_drop)

#Plotting Wordcloud for handles
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title('WordCloud for the Canadian Election hashtags')
plt.axis('off')
plt.show()

It can be seen that hashtags like 'ndp', 'voteppc2021', 'ppc', 'purplereign', 'conservative','singh' etc. provide a good direction for the selection of different set of words assignment.

##**2.2 Party Affiliation of a Tweet**
 
Both the wordclouds helped to identify keywords relevant to a particular party. Only few keywords were listed in this study. Looking at the tweets and doing some research on the 2021 canadian elections, the tweets can be classified into Conservatives, Liberals,PPC and NDP based on commonly used hastags and names of politicians associated with different parties.

In [ ]:
#List of words that identify liberal
liberal = ['liberal', 'liberalparty', 'forwardforeveryone','pm', 'government', 
           'lpc', 'justin', 'trudeau', 'justintrudeau', 'jt', 'cafreeland']

#List of words that identify PPC
PPC = ['ppc', 'peoplespartyofcanada', 'maximebernier', 'peoplespca', 'purple',
       'peoplesparty', 'maxime', 'bernier', 'strongandfree' ]

#List of words that identify NDP
NDP = ['ndp', 'newdemocraticparty', 'newdemocratic', 'jagmeetsingh', 'fightingforyou'
       'jagmeet', 'singh', 'thejagmeetsingh', 'initforyou' ]

#List of words that identify CPC
CPC = ['cpc_hq', 'conservative', 'conservativeparty', 
       'cpc', 'erin', 'otoole', 'erinotoole', 'codyotoole', 'kenney', 'jkenney']

election_df_cp['party'] = 'None' ## initiating an empty colum in the dataframe to be filled with political parties

  *First all the handles and hashtages are removed and new cloumn is saved as 'clean text' before moving ahead in the assignment.

In [ ]:
#Cleaning canadian election data
election_df_cp['clean_text'] = election_df_cp['text'].apply(lambda x: cleaned_data(x,rmove_handle_tag= True))

Assigning tweets to a political party according to the keywords dictionay defined above. Only the unique mentions are assigned, in case there are multiple keywords for a particular tweet it is handled as 'Mixed' 

In [ ]:
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(liberal)),"party"]= "Liberal"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(PPC)),"party"]= "PPC"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(NDP)),"party"]= "NDP"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(CPC)),"party"]= "CPC"

As mentioned above some tweets contain mentions of both the parties hence taking them into consideration as mixed tweets.

In [ ]:
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(liberal)) & election_df_cp['text'].str.contains('|'.join(CPC)),"party"] = "Mixed"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(liberal)) & election_df_cp['text'].str.contains('|'.join(NDP)),"party"] = "Mixed"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(liberal)) & election_df_cp['text'].str.contains('|'.join(PPC)),"party"] = "Mixed"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(CPC)) & election_df_cp['text'].str.contains('|'.join(NDP)),"party"] = "Mixed"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(CPC)) & election_df_cp['text'].str.contains('|'.join(PPC)),"party"] = "Mixed"
election_df_cp.loc[election_df_cp['text'].str.contains('|'.join(NDP)) & election_df_cp['text'].str.contains('|'.join(PPC)),"party"] = "Mixed"

In [ ]:
election_df_cp.head()

In [ ]:
#Inspecting results of classification
election_df_cp[["text","sentiment", "party"]].head(20)

In [ ]:
election_df_cp["party"].value_counts()

**The graph below shows the responses for each political party**

In [ ]:
##distribution of political parties 
plt.figure(figsize=(15,5))

total_entries = election_df_cp.shape[0]
frequency_percent = [i/total_entries*100 for i in election_df_cp['party'].value_counts()]
cum_frequency_percent = np.cumsum(frequency_percent) #calculation of cumulative value

plt.subplot(1,2,2)
plt.yticks(np.arange(0,105,5), rotation='horizontal')
plt.plot(election_df_cp['party'].value_counts().index.tolist(), cum_frequency_percent, 'r-o', 
         label='Cumulative frequency of political parties')
plt.title("Cumulative Frequency of political parties", fontsize = 15)
plt.xlabel("Political Party", fontsize = 15)
plt.ylabel('Cumulative %', fontsize = 15)
plt.legend()

# exploring the profession of responders:
plt.subplot(1,2,1)
ax = election_df_cp['party'].value_counts().plot(kind='bar')
ax.set_title('Political party distribution', fontsize = 15)
ax.set_xlabel("Political Party", fontsize = 15)
ax.set_ylabel("Count of Response", fontsize = 15)
plt.show()

From the above graph, it can be noted that the distribution of tweets are uneven. Libral party has the most count 298, the next indivial mentions is for the CPC party with 169 responses followed by PPC. NDP has the least meantionsof about 20. 

##**2.3 Canadian Election Data Visualization</font>**

### **Canadian Election Dataset**

One of the goals of this assignment is to identify the sentiment of the tweet from the canadian election dataset. There are two kinds of sentiments: 'negative' and 'positive'. 

To have a better understanding of both the sentiments, data is pivoted to sperate party and the sentiment for each of them.
Below is the result of the pivot table that has the distribution of both sentiment.

In [ ]:
##  using pivot tables
election_df_pivot1 = election_df_cp[['sentiment', 'party']].copy()
election_df_pivot1['count'] = 1

## using pivot table to filter out desired columns
election_df_pivot1 = pd.pivot_table(election_df_pivot1, values = 'count', index = 'party',columns = 'sentiment', aggfunc=np.sum, fill_value=0)
election_df_pivot1

To have a even better comparison, a stacked barplot is plotted below.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
sns.set(font_scale=1.2)
# plt.subplot(2,1,1)
ax = election_df_pivot1.loc[['Liberal','Mixed','None', 'CPC','NDP','PPC']].plot.bar(stacked=True, ax = ax1)
plt.xticks(rotation='vertical', fontsize="10")
ax.set_title("Distribution of tweets with political parties and sentiments", fontsize = 14)
ax.set_xlabel("Political Parties", fontsize = 14)
ax.set_ylabel("Count of Tweets", fontsize = 14)


## Plots for Cumulative frequency of tweets for parties
total_entries1 = election_df_cp.shape[0]
frequency_percent1 = [i/total_entries1*100 for i in election_df_cp['party'].value_counts()]
cum_frequency_percent1 = np.cumsum(frequency_percent1) #calculation of cumulative value

plt.plot(election_df_cp['party'].value_counts().index.tolist(), cum_frequency_percent1, 'r-o', label='Cumulative frquency of Political Party Tweets')
plt.xlabel('Political Parties', fontsize = 14)
plt.ylabel('Cumulative Percentage', fontsize = 14)
plt.xticks(rotation='vertical')
plt.title("Cumulative Frequency of political party tweets", fontsize = 14)
plt.legend()
plt.show()

## Plots for distribution of negative tweets
plt.figure(figsize = (8, 6), facecolor = None)
ax = sns.histplot(data=election_df_cp, x='negative_reason', shrink=0.5,)
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right', fontsize='x-small')
plt.ylabel("Count of Tweets", fontsize = 14)
plt.title('Distribution of Negative Reasons');

plt.show()

**Observation:**
It is observed Liberal party has the most tweets count, but the number of negative sentiment tweets is higher than the positive sentiment tweets. While for rest of the groups positive sentiment is higher than the negative sentiment.
NDP though having least count has only one negative tweet.

To further analyse the breakup of negative sentiment, distribution is plotted that shows several categories or the reasons for negative sentiment. Few of the segnificant onces are others, calling early election and lies.
This is further discussed in the last part of the assignment.

**Wordcloud for election sentiment** is employed to understand what kind of words are present in the negative and positive tweets. The below code separates the negative and positive tweet and then presents two plots one for each sentiment. 

In [ ]:
## WordCloud for Positive tweets in Sentiments Dataset

wordcloud_elec_pos = WordCloud(random_state=21, max_font_size=30, background_color ='lightgreen', colormap='Set1',
                               min_font_size = 4).generate(str(election_df_cp[election_df_cp['sentiment']=='positive']['clean_text']))
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 6), facecolor = None) 
plt.imshow(wordcloud_elec_pos) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.title('Positive Sentiments wordcloud of Canadian Elections 2021', fontsize = 14, y = 50)
plt.show()

## WordCloud for NEGATIVE tweets in Sentiments Dataset

wordcloud_elec_neg = WordCloud(random_state=21, max_font_size=30, background_color = 'salmon',
                               min_font_size = 4).generate(str(election_df_cp[election_df_cp['sentiment']=='negative']['clean_text']))


# plot the WordCloud image                        
plt.figure(figsize = (8, 6), facecolor = None) 
plt.imshow(wordcloud_elec_neg) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.title('\nNegative Sentiments wordcloud of Canadian Elections 2021', fontsize = 14, y = 50)
plt.show()

**Observation:**
1. Positive Word Cloud of Elections = Besides 'election' and 'vote' appearing in the positive word cloud, words like 'justintrudeau' and 'ppc' also appears in twitter. This suggested that public has a positive outlook for the liberal party. It can also be observed that there is a combination of both the positive and negative words.

2. The negative wordcloud is more inclined towards negative words. Some of the words which can be highlighted from this plot are 'horrible', 'guns' and 'suffering'. Interestingly, it seems Justin Trudue (Liberal Party) and Otoole (Conservative Party) appear most in the negative cloud. Hence this could highlight that peoples hatred have been focused on the two major ruling parties in Canada i.e. Liberals and Conservatives


3. There are a few generic and common words which could have been removed as a stopword such as 'Canadian', 'Election' and 'Canada'. 

## **2.4 Generic Sentiment Dataset**

A similar approach is followed for the generic tweets. This dataset is going to be used as the training set for further steps.

In [ ]:
# Copy  of sentiment dataset to work on
senti_df_cp_r = senti_df_cp.copy()
senti_df_cp_r['text'] = senti_df['text'].apply(lambda x: cleaned_data(x,rmove_handle_tag=True))

# Wordcloud for positive sentiment
wordcloud_senti_pos = WordCloud(random_state=21, max_font_size=30, background_color ='lightgreen', colormap='Set1', 
                               min_font_size = 4).generate(str(senti_df_cp_r[senti_df_cp_r.label==1]['text']))

plt.figure(figsize=(10, 10))
plt.imshow(wordcloud_senti_pos, interpolation="bilinear")
plt.axis('off')
plt.title('WordCloud for generic positive tweets')
plt.show()

# Wordcloud for positive sentiment
wordcloud_senti_neg = WordCloud(random_state=21, max_font_size=30, background_color ='salmon', 
                               min_font_size = 4).generate(str(senti_df_cp_r[senti_df_cp_r.label==0]['text']))

plt.figure(figsize=(10, 10))
plt.imshow(wordcloud_senti_neg, interpolation="bilinear")
plt.axis('off')
plt.title('\nWordCloud for generic negative tweets')
plt.show()

**Observation:**
The positive and negative cloud is presented above shows that there are  higher order of negative words are found in the negative plot. 

# **Task 3: Sentiment Classification - Model implementation and tuning** 

This task is divided into 5 sections: 


## 3.a Machine learning models: (5 marks)
Steps included are:

1. Splitting the generic tweets into test and train dataset. 80% of the data is used as training set and the remaining data is considered as a test dataset.

2. The original text is than vectorized using TF-IDF Vectorizer.

3. The vectorizer is fitted using only the training data. The fitted vectorizer is later used to find the vectorized output for the other sets using the tranform option.
4. 7 Machine learning Classifier models are trained to predict results on test dataset

### 3.a.1 Split the Dataset

In [ ]:
# Splitting the dataset
X = senti_df_cp_r
y = senti_df_cp_r['label']

#Splitting the dataset 
X_train_1, X_test, y_train_1, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=100)

print("Train Set X, y: {},{}".format(X_train_1.shape, y_train_1.shape))
print("Test Set X, y: {}, {}".format(X_test.shape, y_test.shape))

### 3.a.2 TF-DIF Vectorizer

While computing TF, all terms are considered equally important. However it is known that certain terms, such as “is”, “of”, and “that”, may appear a lot of times but have little importance. Thus it is need to weigh down the frequent terms while scale up the rare ones, by computing IDF, an inverse document frequency factor is incorporated which diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.
IDF is the inverse of the document frequency which measures the informativeness of term t.


Source: https://towardsdatascience.com/tf-term-frequency-idf-inverse-document-frequency-from-scratch-in-python-6c2b61b78558

In [ ]:
#Defining the TFDIF Vectorizer
vectorizer_tf = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, max_features=5000)
vectorizer_tf.fit(X_train_1['text'].values)    # fit data on training sample

#Tranform the train and validation sets
X_train_tf = vectorizer_tf.transform(X_train_1['text'].values)
X_test_tf = vectorizer_tf.transform(X_test['text'].values)

print("Shape after tfdif vectorizations")
print("Train Set X, y: {},{}".format(X_train_tf.shape, y_train_1.shape))
print("Test Set X, y: {}, {}".format(X_test_tf.shape, y_test.shape))

### **3.a.3 Classifier Models</font>**

This section focuses on using the prepared data as the input to the 7 different models with different classification algorithms: 

1. Logistic Regression
2. Naive Bayes
3. SVM
4. Decision Tree
5. Random Forest
6. KNN
7. XG-Boost

The below cell implements a generalized function for each iteration of models runs.

In [ ]:
#function to implement model training
def implement_model(model, X, y, Xtest, ytest, check=True):

  X_trn = X
  y_trn = y

  #Model Fit
  model.fit(X_trn,y_trn)

  #Train Predictions
  y_pred_trn = model.predict(X_trn)

  #check this for the case when XGBClassifier is used.
  if check==False:
    y1 = [round(value) for value in y_pred_trn]
    #y2 = [round(value) for value in y_pred_val]
    y_pred_trn = y1
    #y_pred_val = y2
  
  #Calculate Accuracies
  acc_trn = metrics.accuracy_score(y_trn, y_pred_trn)

  #Test Accuracies
  y_preds = model.predict(Xtest)
  acc_test = metrics.accuracy_score(y_test, y_preds)
  report_test = classification_report(y_test,y_preds)

  #Print Accuracies for comparison
  print("The accuracy score for the Train set is: {0:.3}".format(acc_trn))
  print("The accuracy score for the Test set is: {0:.3}".format(acc_test))
  print("\nThe classification report for the Test set is :\n {}".format(report_test))

  return acc_trn, acc_test, report_test

#### **Model 1: Logistic Regression**

Logistic regression is a statistical model that in its basic form uses a logistic function to model a binary dependent variable, although many more complex extensions exist. In regression analysis, logistic regression (or logit regression) is estimating the parameters of a logistic model (a form of binary regression).

Source: https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303bc?gi=9e2a67956d00

In [ ]:
model_1 = LogisticRegression(max_iter=10000)

print("\nResults for Tf-idf Vectorizer for Logistic Regression model:\n")
logistic_model = implement_model(model_1, X_train_tf, y_train_1, X_test_tf, y_test)

####**Model 2: Naive Bayes (Multinomial)**

nave Bayes classifiers are a type of "probabilistic classifier" based on Bayes' theorem and strong (nave) independence assumptions between features.
Models for assigning class labels to problem instances that are represented as vectors of feature values, with the class labels taken from a finite set. For training such classifiers, there is no one algorithm, but rather a variety of algorithms based on the same principle: all naive Bayes classifiers assume that the value of one feature is independent of the value of any other feature, given the class variable. For example, if a fruit is red, round, and around 10 cm in diameter, it is termed an apple. Each of these characteristics is taken into account by a naive Bayes classifier, regardless of any possible correlations between the color, roundness, and diameter features.

Source : https://www.analyticsvidhya.com/blog/2021/01/a-guide-to-the-naive-bayes-algorithm/

In [ ]:
model_2 = MultinomialNB()

print("\nResults for Tf-idf Vectorizer for Naive Bayes (Multinomial) model:\n")
naive_bayes_model = implement_model(model_2, X_train_tf, y_train_1, X_test_tf, y_test,)

####**Model 3: SVM (Linear SVC)**
LinearSVC uses a multi-class technique of "one vs. the others" to train n class models. Only one model is trained if there are only two classes. They are supervised learning models that evaluate data for classification and regression analysis, as well as associated learning algorithms. An SVM training algorithm creates a model that assigns new examples to one of two categories, making it a non-probabilistic binary linear classifier, given a series of training examples that are individually designated as belonging to one of two categories (although methods such as Platt scaling exist to use SVM in a probabilistic classification setting).

SVMs may perform non-linear classification as well as linear classification by implicitly mapping their inputs into high-dimensional feature spaces, which is known as the kernel trick.

Source: https://pythonprogramming.net/linear-svc-example-scikit-learn-svm-python/

In [ ]:
model_3 = LinearSVC(max_iter=10^6)

print("\nResults for Tf-idf Vectorizer for LinearSVC (SVM) model:\n")
linear_SVC_model = implement_model(model_3, X_train_tf, y_train_1, X_test_tf, y_test)

####**Model 4 Decision Trees**

A decision tree is a decision-making aid that employs a tree-like model of decisions and their potential results, such as chance event outcomes, resource costs, and utility. It's one approach to show an algorithm made up entirely of conditional control statements.

Decision trees are a popular technique in machine learning and are often used in operations research, specifically in decision analysis, to help determine the best method for achieving a goal.

Source: https://www.datacamp.com/community/tutorials/decision-tree-classification-python

In [ ]:
model_4 = DecisionTreeClassifier()

print("\nResults for Tf-idf Vectorizer for DecisionTree model:\n")
decision_tree_model = implement_model(model_4, X_train_tf, y_train_1, X_test_tf, y_test)

####**Model 5 Random Forests**
Random forests, also known as random decision forests, are an ensemble learning method for classification, regression, and other tasks that works by training a large number of decision trees and then outputing the class that is the mode of the classes (classification) or the mean prediction (regression) of the individual trees. Random decision forests address the problem of decision trees overfitting their training set.

Source: https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76

In [ ]:
model_5 = RandomForestClassifier()

print("\nResults for Tf-idf Vectorizer for Random Forest Classifier model:\n")
RandomForest_model = implement_model(model_5, X_train_tf, y_train_1, X_test_tf, y_test)

#### **MODEL 6: k-Nearest Neighbours (kNN)Classifier**

The outcome of k-NN classification is a class membership. An object is categorised by a majority vote of its neighbours, with the object allocated to the most common class among its k closest neighbours (k is a positive integer, typically small). If k = 1, the item is simply assigned to that single nearest neighbor's class.

Source: https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761

In [ ]:
model_6 = KNeighborsClassifier()

print("\nResults for Tf-idf Vectorizer for k-Nearest Neighbours Classifier model:\n")
kNN_model = implement_model(model_6, X_train_tf, y_train_1, X_test_tf, y_test)

#### **MODEL 7: XGBosst Classifier**
XGBoost is an implementation of gradient boosted decision trees designed for speed and performance that is dominative competitive machine learning.

Source: https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/

In [ ]:
model_7 = XGBClassifier()

print("\nResults for Tf-idf Vectorizer for XGBosst Classifier model:\n")
XGBClassifier_model = implement_model(model_7, X_train_tf, y_train_1, X_test_tf, y_test, check=False)

###3.a.4 Models Analysis

In [ ]:
#test cell
fig, ax = plt.subplots(1,1,figsize=(15,8))

for i in range(2):
  if i == 0:
    acc_trn_scores= [logistic_model[i], naive_bayes_model[i],linear_SVC_model[i],decision_tree_model[i],
                   RandomForest_model[i],kNN_model[i],XGBClassifier_model[i]]
  elif i == 1:
    acc_test_scores= [logistic_model[i], naive_bayes_model[i],linear_SVC_model[i],decision_tree_model[i],
                   RandomForest_model[i],kNN_model[i], XGBClassifier_model[i]] 

classifiers_list = ['Logistic regression', 'Naive Bayes', 'SVM', 'Decision Trees', 'Random Forests','KNN', 'XG Boost'] #

plt.subplot(1,1,1)

plt.plot(classifiers_list, acc_trn_scores, marker='o', color='y', linewidth=3, label = 'Accuracy train')

plt.plot(classifiers_list, acc_test_scores, marker='o', color='b',linestyle='-.', linewidth=3, label = 'Accuracy test')

plt.xticks(np.arange(0,7,1), rotation=0, fontsize=12)
plt.xlabel('Classifier Models', fontsize=15)
# plt.ylim([0.6,0.8])
plt.ylabel('Accuracy Score', fontsize=15)
plt.title('TFIDF Model Accuracies across classifiers')


ymax = max(np.asarray(acc_trn_scores, dtype=np.float32))
xmax = np.where(acc_trn_scores == np.amax(np.asarray(acc_trn_scores)))[0]
plt.annotate('Max Acc. train = (%.5f)'%(np.amax(np.asarray(acc_trn_scores))), 
             xy=(xmax, ymax), xytext=(xmax-1.5, ymax+0.0005), 
             arrowprops=dict(facecolor='y', shrink = 0.005, frac=0.1)
             )
             
ymax = max(np.asarray(acc_test_scores, dtype=np.float32))
xmax = np.where(acc_test_scores == np.amax(np.asarray(acc_test_scores)))[0]
plt.annotate('Max Acc. test = (%.5f)'%(np.amax(np.asarray(acc_test_scores))), 
             xy=(xmax, ymax), xytext=(xmax+0.5, ymax+0.010), 
             arrowprops=dict(facecolor='b', shrink = 0.005, frac=0.1)
             )

plt.legend()
plt.show()

**Observation**

It can be clearly seen from the above graph that the best model that priovided the best accuracy would be Logistic regression. The performance for that model on the test set is 0.965. Performance of two other models (Naive Bayes and SVM) are close to logistic regression.
It is worthy to note that the decision tree classifers have best training accuracy by the test accuracy is not as good as training accuracy. On of the reason for this observation would be overfitting of the data. This is commonly observed in decision trees if the hyperparameters are not tuned. In this case, general model is used without hyperparameter tuning.
XGboost had the least training and test accuracy.

So, amongh all the models Logistic regression is preffered for this classification problem as it has th ebest train and test accuracy along with less computation time.

## 3.b Deep learning model: (4 marks)

Steps similar to above part is applied.
1. Splitting the generic tweets into train (60%),validation(20%) and test(20%)dataset

2. The original text is then vectorized using Bag of Words (TF).

3. The vectorizer is fitted using only the training data. The fitted vectorizer is later used to find the vectorized output for the other sets using the tranform option.
4. Deeplearing models are trained to predict results on validation and test dataset

### 3.b.1 Split the Dataset

In [ ]:
# Splitting the dataset further using split in part 3.a.1 in which data was split into Training 80% and Test 20%

#Splitting the dataset 
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train_1, y_train_1, test_size=0.25, stratify=y_train_1, random_state=100)

###**3.b.2 Word Frequency Data Preparation (Bag of Words) or Term Frequency (TF)**

Bag of Words is a method to extract features from text documents. These features can be used for training machine learning algorithms. It creates a vocabulary of all the unique words occurring in all the documents in the training set. 

It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

Source: https://machinelearningmastery.com/gentle-introduction-bag-words-model/

In [ ]:
#Defining the Vectorizer
vectorizer_bag = CountVectorizer(max_features = 2500)
vectorizer_bag.fit(X_train_2['text'].values)

#Transforming Data
X_train_bag = vectorizer_bag.transform(X_train_2['text'].values)
X_val_bag = vectorizer_bag.transform(X_val['text'].values)
X_test_bag = vectorizer_bag.transform(X_test['text'].values)

print("Shape after count vectorizations")
print("Train Set X, y: {},{}".format(X_train_bag.shape, y_train_2.shape))
print("Validation Set X, y: {},{}".format(X_val_bag.shape, y_val.shape))
print("Test Set X, y: {}, {}".format(X_test_bag.shape, y_test.shape))

### **3.b.3 Deeplearning Models**


#### Deeplearning MPL classifier model

MLPClassifier stands for Multi-layer Perceptron Classifier, which is linked to a Neural Network by its name. Unlike other classification methods such as Support Vectors or Naive Bayes Classifier, MLPClassifier does classification using an underlying Neural Network.

However, MLPClassifier is identical to Scikit-other Learn's classification algorithms in that it requires no more effort to implement than Support Vectors, Naive Bayes, or any other Scikit-Learn classifier. 

Although, the MLPClassifier using the hidden Neural network make the increases computation time.

In order limit the time, only 2500 features is selected for the model. This might led to lesser accuracy in the model but is adopted in this assignment, due to computaion limitation. 

Source : https://analyticsindiamag.com/a-beginners-guide-to-scikit-learns-mlpclassifier/

In [ ]:
clf_model = MLPClassifier(hidden_layer_sizes = 100, batch_size=512, 
                          learning_rate_init = 0.01, shuffle = True, 
                          verbose = False, random_state=1, max_iter=50
                          ).fit(X_train_bag, y_train_2)

#Model Fit                          
clf_model.fit(X_train_bag, y_train_2)

#Train Predictions
y_pred_trn_1 = clf_model.predict(X_train_bag)

#Validation Predictions
y_val_pred_1 = clf_model.predict(X_val_bag)

#Calculate Accuracies
acc_train_1 = metrics.accuracy_score(y_train_2, y_pred_trn_1)
acc_val_1 = metrics.accuracy_score(y_val, y_val_pred_1)

print('The training accuracy without hyperparameter tuning is: ', acc_train_1)
print('The validation accuracy without hyperparameter tuning is: ', acc_val_1)

Results from the above model is comparable with the ML models implemented earlier with Training accuracy (0.993) and test accuracy (0.952).

In the next step, hyperparameter is tuned usinig grid search.
Here, two parameter hidden_layer_sizes is tuned along with the learning rate.

Few other parameters that can be trained include the activation function and also the level of neural networks. It is not adpoted in this assignment due to long running time and heavy computation.

In [ ]:
# Hypertuning parameters using gridsearch
param = {'hidden_layer_sizes': [30,50,100],
         'learning_rate_init':[0.1, 0.01],
         #'activation': ['tanh', 'relu'],          # Removed due to heavy computation
         }
         
MLP_model = MLPClassifier(batch_size= 1024,max_iter= 50, random_state=128)
MLP_model_gridsearch = GridSearchCV(MLP_model, param, verbose=0, scoring='accuracy')

#Model Fit
MLP_model_gridsearch.fit(X_train_bag, y_train_2)

#Train Predictions
y_pred_trn_2 = MLP_model_gridsearch.predict(X_train_bag)

#Validation Predictions
y_val_pred_2 = MLP_model_gridsearch.predict(X_val_bag)

#Calculate Accuracies
acc_train_2 = metrics.accuracy_score(y_train_2, y_pred_trn_2)
acc_val_2 = metrics.accuracy_score(y_val, y_val_pred_2)

#Best model
MLP_model_gridsearch_bs = MLP_model_gridsearch.best_score_
MLP_model_gridsearch_bp = MLP_model_gridsearch.best_params_

print('The training accuracy hyperparameter tuning is: ', acc_train_2)
print('The validation accuracy hyperparameter tuning is: ', acc_val_2)
print('\n Best Tuning Score is =', MLP_model_gridsearch_bs)
print('\n Best Parameters are =', MLP_model_gridsearch_bp)

**Observation**

Above are the results from gridsearch : best parameters are {'hidden_layer_sizes': 100, 'learning_rate_init': 0.01} with test accuracy of (0.954)


#### **Deeplearning Nerural network with Pytorch**

The deep learning model is implemented with Pytorch.
Steps involved:
1. X_train, X_val  and X_test data that were transformed using Countvectorizer is now converted to torch.tensor, that be used as an input for the model.

2. All the X_data Data is loaded using DataLoader as an input to the neural network.
3. Model parameters are defined with loss function, epoch and learning rate.
4. Model training
5. Prediction on y_val.
6. Hyperparameter training for best model parameters.
7. Preaction on testset with the best model parameters.





In [ ]:
# Define a function to convert x_csr matrix to torch matrix
def convert_torch(X_data):

  coo = coo_matrix((X_data), shape=(X_data.shape[0], X_data.shape[1]))

  values = coo.data
  indices = np.vstack((coo.row, coo.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = coo.shape

  X_data_torch = torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

  return X_data_torch

In [ ]:
X_train_bag_torch = convert_torch(X_train_bag) # Convert train data to torch
X_val_bag_torch = convert_torch(X_val_bag)     # Convert val data to torch
X_test_bag_torch = convert_torch(X_test_bag)   # Convert train data to torch

type(X_train_bag_torch),type(X_val_bag_torch),type(X_test_bag_torch)

### Modeling : **Deep Learning Model**

In [ ]:
#defining dataset class

class train_val_data(Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]

  def __len__(self):
    return len(self.x)

train_torch = train_val_data(X_train_bag_torch , torch.FloatTensor(y_train_2.to_numpy()))
val_torch = train_val_data(X_val_bag_torch , torch.FloatTensor(y_val.to_numpy()))

## test data    
class test_data(Dataset):
    
    def __init__(self, x):
      self.x = x
        
    def __getitem__(self, idx):
        return self.x[idx]
        
    def __len__ (self):
        return len(self.x)
    
test_torch = test_data(X_test_bag_torch)

In [ ]:
# Model parameters
epochs = 10
b_size = 160
learning_rate = 0.01

In [ ]:
#DataLoader
train_loader = DataLoader(train_torch, batch_size= b_size)
val_loader = DataLoader(val_torch, batch_size=b_size)
test_loader = DataLoader(test_torch, batch_size=1)

In [ ]:
#Define the neural network

class classification(nn.Module):
  def __init__(self,input_shape):
    super(classification,self).__init__()
    self.fc1 = nn.Linear(input_shape,64)
    self.fc2 = nn.Linear(64,32)
    self.fc3 = nn.Linear(32,1)

  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
# nnModel 
model_nn = classification(input_shape= 2500)
optimizer = torch.optim.SGD(model_nn.parameters(),lr=learning_rate)
loss_function = nn.BCEWithLogitsLoss()

In [ ]:
# Define accuracy function
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
# Model training 
model_nn.train()
for epoch in tqdm(range(0, epochs)):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        y_pred = model_nn(X_batch)
        
        loss = loss_function(y_pred, y_batch.unsqueeze(1))
        acc = accuracy(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    print(f'Epoch {epoch+ 1}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

In [ ]:
# Prediction on validation data
list_y_pred = []
model_nn.eval()
with torch.no_grad():
    for X_batch,y_batch in val_loader:
        y_val_pred = model_nn(X_batch)
        y_val_pred = torch.sigmoid(y_val_pred)
        y_pred_tag = torch.round(y_val_pred)
        list_y_pred.append(y_pred_tag.cpu().numpy())

list_y_pred = [a.squeeze().tolist() for a in list_y_pred]

In [ ]:
y_pred_list_temp = []
for i in list_y_pred:
  for elements in i:
    y_pred_list_temp.append(elements)
y_pred_list = y_pred_list_temp
report_nn = classification_report(y_val, y_pred_list)
print("\nThe classification report for the Validation set is :\n {}".format(report_nn))

In [ ]:
#Hyperparameter tuning
best_params_hp = {}
best_accuracy_hp = 0


for learning_rate_hp in tqdm([0.1,0.01,0.001]):

    for b_size_hp in [32,64,128,256]:

      train_loader_hp = DataLoader(train_torch, batch_size= b_size_hp)
      val_loader_hp = DataLoader(val_torch, batch_size = b_size_hp)
      
      model = classification(input_shape= 2500)
      optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate_hp)
      loss_fn = nn.BCEWithLogitsLoss() 

      model.train()
      for X_batch, y_batch in train_loader_hp:
        
        optimizer.zero_grad()
        
        y_pred_hp = model(X_batch)
        
        loss = loss_fn(y_pred_hp, y_batch.unsqueeze(1))
        acc = accuracy(y_pred_hp, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

      y_pred_list_hp = []
      model.eval()
      with torch.no_grad():
          for X_batch,y_batch in val_loader_hp:
             
              y_val_pred_hp = model(X_batch)
              y_val_pred_hp = torch.sigmoid(y_val_pred_hp)
              y_pred_tag_hp = torch.round(y_val_pred_hp)
              y_pred_list_hp.append(y_pred_tag_hp.cpu().numpy())

      y_pred_list_hp = [a.squeeze().tolist() for a in y_pred_list_hp]
      
      y_pred_list_temp_hp = []
      for i in y_pred_list_hp:
        for element_hp in i:
          y_pred_list_temp_hp.append(element_hp)
      y_pred_list_new_hp = y_pred_list_temp_hp

      con_matrix_hp = confusion_matrix(y_val, y_pred_list_new_hp)
      #res = cm.tolist()
      accuracy_nn_hp = (con_matrix_hp[0][0] + con_matrix_hp[1][1] ) * 100/ (con_matrix_hp[0][0] + con_matrix_hp[1][0] + con_matrix_hp[0][1] + con_matrix_hp[1][1] )
      print("Accuracy for learning rate:",learning_rate_hp, ' and batch size:',b_size_hp,' is:', accuracy_nn_hp)

      if accuracy_nn_hp > best_accuracy_hp:
        best_params_hp = {'Learning rate':learning_rate_hp, 'Batch size':b_size_hp, 'model' : model}
        best_accuracy_hp = accuracy_nn_hp

print ('\n','Best Parameters are:','\n',best_params_hp)

print ('\n',"Best Score: {0:.3}".format(best_accuracy_hp))   

In [ ]:
#Testing model using tuned parameters

y_pred_test_list = []
model = best_params_hp.get('model')
model.eval()
with torch.no_grad():
    for X_batch in test_loader:

        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_test_list.append(y_pred_tag.cpu().numpy())

y_pred_test_list = [a.squeeze().tolist() for a in y_pred_test_list]

#Test Accuracies
acc_test_nn = metrics.accuracy_score(y_test, y_pred_test_list)
report_test_nn = classification_report(y_test,y_pred_test_list)

#Print Accuracies for comparison
print("The accuracy score for the Test set is: {0:.3}".format(acc_test_nn))
print("\nThe classification report for the Test set is :\n {}".format(report_test_nn))


## **3.c Implementing Final Model on elections dataset: (0.5 marks)**

Based on the performance of all the models in part 3.a and 3.b, logistics regression performed the best. Deep learning model also shows good results, inorder to keep the computation time in check, logistic regression model is selected for prediction on Canadian election dataset


In [ ]:
## Mapping positive and negative to 1 and 0 in the Canadian elections dataset
map_sentiment = {'positive':1, 'negative':0}

X_election = election_df_cp['clean_text']
y_election = election_df_cp.replace({'sentiment': map_sentiment})['sentiment']

X_election_tf = vectorizer_tf.transform(X_election)

In [ ]:
#Best model Logistic Regression is used to predict the election results

model_final = LogisticRegression(max_iter=10000)

#Model Fit
model_final.fit(X_train_tf,y_train_1)

#Test Predictions
y_pred_test = model_final.predict(X_test_tf)

#Calculate Accuracies
acc_test_model = metrics.accuracy_score(y_test, y_pred_test)
  
print("The accuracy score for the Test (Generic dataset) is: {}".format(acc_test_model))



In [ ]:
#Test and prediction Accuracies on Canadian Election dataset
y_preds_elect = model_final.predict(X_election_tf)  ## Using this for further prediction

acc_test_elect = metrics.accuracy_score(y_election, y_preds_elect)
report_test_elect = classification_report(y_election,y_preds_elect)

#Print Accuracies for comparison
print("The accuracy score for the Test (Election Dataset) is: {0:.3}".format(acc_test_elect))
print("\nThe classification report for the Test (Election Dataset) is :\n {}".format(report_test_elect))


**Observation**

Logisticregression model achieved accuracy of 0.714. Accuracy drop is clear as compared with the generic dataset where test accuracy was 0.966.
There could be multiple reasons for lower accuracy, one being the features in both the datasets.

As observed in the wordclouds for negative election datasets many keywords related to political parties was seen e.g, 'trudue' (Liberal Party) and 'otoole' (Conservative Party). These keywords have very less probalilty occurances in the generic dataset used to train the model. Same is applicable with the positive sentiment keywords in both the datasets.

## **3.d Other evaluation metrics**

Evalution metric used so far is accuracy_score [sum of diagonal elements (true positive and true negative)/ all elements] of the confusion matrix. Since, the assignment deals with binary classification of positive and negative sentiments is a good matrix for evalution considering distribution of the dataset roughly (2/3 positive and 1/3 negative).
Incase dataset is even more unbalanced two matrix that could be applied are.
1. F1 score: This matrix is best for highly imbalanced data, it involves presion and recall to compute the F1 score, which ensures better handling of misclassification.
**Note**: These scores are already plotted with the results of the model as classification, but the model selection is done based on accuracy_score

2. An ROC curve (receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. This curve plots two parameters:True Positive Rate, False Positive Rate
It an graphical representation to have comparision among different models.

## **3.e Visualize the sentiment prediction results**

In [ ]:
## Making a Dataframe of True and False prediction on Canadian elections dataset
import copy
election_df_cp_2 = copy.deepcopy(election_df_cp)
election_df_cp_2 = election_df_cp_2.drop(columns = ['handles','hashtags'])
election_df_cp_2['sentiment_predictions'] = pd.Series(y_preds_elect).map({0:'negative',1:'positive'})

election_df_cp_2['sentiment_vs_pred'] = 0

for i in range (election_df_cp_2.shape[0]):
    if election_df_cp_2['sentiment'][i] == election_df_cp_2['sentiment_predictions'][i]:
        election_df_cp_2['sentiment_vs_pred'][i] = 'Correct_Prediction'
    else:
        election_df_cp_2['sentiment_vs_pred'][i] = 'Wrong_Prediction'

In [ ]:
election_df_cp_2.head()

In [ ]:
#Plot Correct vs Wrong Predictions for each party 
plt.figure(figsize=(12,7))
party_order = ['Liberal','Mixed','None', 'CPC','NDP','PPC']
ax = sns.countplot(x="party", data=election_df_cp_2, hue='sentiment_vs_pred', order= party_order)
ax.set_title('Correct vs Wrong Predictions for each party', fontsize = 20)
ax.set_xlabel('Political Party', fontsize = 17)
ax.set_ylabel('Count of Predictions', fontsize = 17)
plt.show()

In [ ]:
#Plot 
plt.figure(figsize=(20,4*2))
gs = gridspec.GridSpec(1,2)

plt.subplot(gs[0])
ax1 = sns.countplot(x="party", data=election_df_cp_2[election_df_cp_2['sentiment_vs_pred'] == 'Correct_Prediction'], hue='sentiment_predictions',order= party_order)
ax1.set_title('Correct Sentiment Predictions Political Distribution', fontsize = 20)
ax1.set_xlabel('Political Party', fontsize = 15)
ax1.set_ylabel('Count of Predictions', fontsize = 15)


plt.subplot(gs[1])
ax2 = sns.countplot(x="party", data=election_df_cp_2, hue='sentiment_predictions',order= party_order)
ax2.set_title('Overall Sentiment Predictions Political Distribution', fontsize = 20)
ax2.set_xlabel('Political Party', fontsize = 15)
ax2.set_ylabel('Count of Predictions', fontsize = 15)


plt.show()
plt.tight_layout()

display(Markdown('\n---------------------------------- Model Result ----------------------------------\n'))
print('Accuracy = ', "{0:.3%}".format(acc_test_elect))

**Observation:**

It is evident from the graphs that three political parties and the mixed set have quite a lot negative sentiments expressed in tweets. 

Top two parties **Liberal Party** and **Conservative party** have higher proportion of negative sentiments in overall prediction, but the conservative party correct prediction is more or less equal as compared to overall prediction. 

Moreover, the **Liberal Party**, although the party has many negetive tweet, the predicted overall positive to negtive tweet ratio is high. Also, Liberal Party has the maximum number of predicted positive tweets amongst all three political parties.

Additionally, **NDP** has the best predicted positive to negative tweets ratio amongst all the three political parties. But,the total number of twwet for the NDP is quite low to produce any quantifiable and measurbale impact to the predictions

**Model Predicted Result:**

According to the anticipated model outlined in the previous section, the **Liberal Party** should win.

**Actual Election Result 2021:**

The **Liberal Party** won the elections.

**Conclusion**

This demonstrates that tweet **NLP analytics can be a highly effective tool** for predicting election outcomes. The public opinion regarding political parties is represented through tweets, and performing tweet NLP can assist political parties in assessing public sentiment and making appropriate reforms.

# **Task 4: Negative Reason Classification - Model implementation and tuning (3 marks):** 


The focus of this section is on the negative labelled data in the Canadian election dataset. The dataset is then divided into 70 % for training and 30 % for testing.

In [ ]:
election_df_cp_2_negative = election_df_cp_2[election_df_cp_2['sentiment']=='negative'][['negative_reason', 'clean_text']]
election_df_cp_2_negative.head()

In [ ]:
election_df_cp_2_negative['negative_reason'].unique()

In [ ]:
plt.figure(figsize=(20,8))

ax3 = sns.countplot(x="negative_reason", data=election_df_cp_2_negative, order = election_df_cp_2_negative['negative_reason'].value_counts().index)
plt.xticks(rotation='vertical', fontsize="15")
ax3.set_xlabel('Negative Reason', fontsize = '15')
ax3.set_ylabel('Count', fontsize = '20')
ax3.set_title('Count of Negative Reasons with labels as percentage of total',  fontsize = '20')

for p, label in zip(ax3.patches, np.round(np.true_divide(election_df_cp_2_negative['negative_reason'].value_counts().values, election_df_cp_2_negative['negative_reason'].value_counts().sum()),3)):
    ax3.annotate("{0:.1%}".format(label), (p.get_x()+0.15, p.get_height()+0.4))

Observation:
It can be observed from the table above that the data is skewed for the last 5 categories which represent a very low percentage of the total. Hence we can combine a few categories

Categories =

1. Calling election early

2. Exploitation = 'Tell Lies' + 'Scandal' + 'segregation' - Reason being : They all form a part of manipultion by leaders

3. Economy = 'economy' + 'covid' - Reason being : COVID has officially become a reason for downfall of ecomomy

4. Healthcare = 'Women Reproductive right and Racism' + 'Healthcare and Marijuana' - Reason being : They all relate to healthcare and wellness of the public

5. Gun control

6. Climate Problem

7. Others

In [ ]:
election_df_cp_2_neg_comb = copy.deepcopy(election_df_cp_2_negative)

election_df_cp_2_neg_comb['combined_neg_reason'] = election_df_cp_2_neg_comb['negative_reason'].map({'tell lies':'Exploitation',
                                                                                                     'calling election early':'early election',
                                                                                                      'scandal':'Exploitation',
                                                                                                       'segregation':'Exploitation',
                                                                                                        'economy':'Economy',
                                                                                                        'covid ':'Economy',      
                                                                                                        'women reproductive right and racism':'Healthcare',
                                                                                                         'healthcare and marijuana':'Healthcare',
                                                                                                          'climate problem':'Climate Problem', 
                                                                                                           'gun control':'gun control',
                                                                                                           'others':'Others'
                                                                                                      }
                                                                                                    )
election_df_cp_2_neg_comb.head(10)

In [ ]:
plt.figure(figsize=(15,6))

ax4 = sns.countplot(x="combined_neg_reason", data=election_df_cp_2_neg_comb, order = election_df_cp_2_neg_comb['combined_neg_reason'].value_counts().index)
plt.xticks(rotation='vertical', fontsize="15")
ax4.set_xlabel('Negative Reason Combined', fontsize = '20')
ax4.set_ylabel('Count', fontsize = '20')
ax4.set_title('Count of COMBINED Negative Reasons with labels as percentage of total',  fontsize = '20')

for p, label in zip(ax4.patches, np.round(np.true_divide(election_df_cp_2_neg_comb['combined_neg_reason'].value_counts().values, election_df_cp_2_neg_comb['combined_neg_reason'].value_counts().sum()),3)):
    ax4.annotate("{0:.1%}".format(label), (p.get_x()+0.15, p.get_height()+1.5))

**Observation:** The data looks much better now and can be further processed for classification

Implimenting classification models

In [ ]:
## Encoding for multiclass classification
neg_mapping = {'Others':0, 'Exploitation':1, 'early election':2, 'Economy':3,'gun control':4,'Climate Problem':5,'Healthcare':6}

X_elec_comb = election_df_cp_2_neg_comb['clean_text']
y_elec_comb = election_df_cp_2_neg_comb.replace({'combined_neg_reason': neg_mapping})['combined_neg_reason']

X_elec_comb_tr, X_elec_comb_test, y_elec_comb_tr, y_elec_comb_test = train_test_split(X_elec_comb, y_elec_comb, test_size=0.30, stratify=y_elec_comb, random_state=100)

#Use countVectorizer()
vectorizer_tf_2 = CountVectorizer()
vectorizer_tf_2.fit(X_elec_comb)

#Transform the test and the train data using the fitted vectorizer
X_elec_comb_tr_tf = vectorizer_tf_2.transform(X_elec_comb_tr)
X_elec_comb_test_tf = vectorizer_tf_2.transform(X_elec_comb_test)

print("Shape after count vectorizations")
print("Train Set X, y: {},{}".format(X_elec_comb_tr_tf.shape, y_elec_comb_tr.shape))
print("Test Set X, y: {}, {}\n".format(X_elec_comb_test_tf.shape, y_elec_comb_test.shape))

In [ ]:
def implement_model_cv(model, X, y, params, scoring):
  
  param_grid = params

  kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
  clf_grid = GridSearchCV(model, param_grid, scoring=scoring, cv=kfold, return_train_score=True)
  clf_grid.fit(X, y)

  results = pd.DataFrame.from_dict(clf_grid.cv_results_)
  results = results.sort_values(['param_{}'.format(list(params.keys())[0])])
  print("The parameter is: {}".format(['param_{}'.format(list(params.keys())[0])]))
  
  train_score_mean = results['mean_train_score']
  train_score_std = results['std_train_score']

  cv_score_mean = results['mean_test_score']
  cv_score_std = results['std_test_score']

  K = results['param_{}'.format(list(params.keys())[0])]
  plt.plot(K, train_score_mean, label='Train Score')
  plt.plot(K, cv_score_mean, label='CV Score')

  plt.scatter(K, train_score_mean, label='Train Score points')
  plt.scatter(K, cv_score_mean, label='CV Score points')

  plt.xscale('log')
  plt.legend()
  plt.xlabel("K: hyperparameter")
  plt.ylabel("Score")

  plt.title("Hyper parameter Vs Accuracy Score plot")
  plt.grid()
  plt.show()

  error_train = np.add(np.power((1-train_score_mean),2),np.power(train_score_std,2))
  error_test = np.add(np.power((1-cv_score_mean),2),np.power(cv_score_std,2))

  plt.plot(K, error_train, label='Train Total Error')
  plt.plot(K, error_test, label='CV Total Error')

  plt.scatter(K, error_train, label='Train Error points')
  plt.scatter(K, error_test, label='CV Error points')

  plt.xscale('log')
  plt.legend()
  plt.xlabel("K: hyperparameter")
  plt.ylabel("Error")

  plt.title("Hyper parameter Vs Total Error (Bias^2 + Variance)")
  plt.grid()
  plt.show()

  print("\nThe best parameter value is: {}".format(clf_grid.best_params_))
  print("\nThe best score is: {}".format(clf_grid.best_score_))

  return clf_grid.best_score_, clf_grid.best_params_, clf_grid.best_estimator_

### **Selected Model: Logisitic Regression**

Logistic Regression was chosen for this challenge just to compare with the results from the previous section. The overall error is lowest for the C value of 1, as can be seen from the charts below. Despite the fact that the model's performance on the test set is far inferior to that on the training set.

In [ ]:
model_election_neg = LogisticRegression(max_iter=10000, multi_class='multinomial')
C = [0.01, 1, 100, 1000, 10000, 100000]
param_grid = dict(C=C)

print("\nResults when Count Vectorizer input is selected:\n")
best_score, best_param, best_est = implement_model_cv(model_election_neg, X_elec_comb_tr_tf, y_elec_comb_tr, param_grid, 'accuracy')

The final model's hyperparameters are selected set, and the model's performance is evaluated across the entire training set. As can be seen, the model performs admirably. This indicates that there is overfitting in this instance. The dataset is quite small, which makes it difficult to improve the test results if the same dataset is used.

We can see from the hyperparameter tuning section that the cross validation set's accuracy was very low. However, when the entire training dataset is considered, the accuracy is 100 percent. This emphasises the issue of low datapoints.

In [ ]:
model_election_neg = LogisticRegression(max_iter=10000, multi_class='multinomial', C=100)

#Fit the model
model_election_neg.fit(X_elec_comb_tr_tf,y_elec_comb_tr)

#Predict
y_preds_neg_tr = model_election_neg.predict(X_elec_comb_tr_tf)

#Accuracy
accs_neg_tr = metrics.accuracy_score(y_elec_comb_tr, y_preds_neg_tr)

print("The accuracy score for the Train negative tweets: {}".format(accs_neg_tr))

In [ ]:
def visualize(y_true, y_preds):
  f1_cal = metrics.f1_score(y_true, y_preds, average=None)
  cm = confusion_matrix(y_true, y_preds)

  fig,ax= plt.subplots(figsize=(10,5))
  sns.heatmap(cm, annot=True, ax = ax,fmt='g', cmap="YlGnBu");

  # labels, title and ticks
  ax.set_xlabel('Predicted labels');
  ax.set_ylabel('True labels');
  ax.set_title('Confusion Matrix for Canadian Elections Dataset');
  

  #Calculating F1 score for each class using sklearn function
  print("\nThe F1 score for each class is: {}".format(f1_cal))
  plt.show()
  
  can_ori = np.sum(cm, axis=1)
  can_pre = np.sum(cm, axis=0)


  ind = np.arange(7)
  fig,ax= plt.subplots(figsize=(10,5))
  rects1 = ax.bar(ind, can_ori, width=0.1, color='blue', label='True Class')
  rects2 = ax.bar(ind+0.1, can_pre, width=0.1, color='green', label='Predicted Class')
  labels= ('Others', 'Exploitation', 
              'early election', 'Economy', 'gun control',
              'Climate Problem', 'Healthcare')
  plt.xticks(ind, labels,rotation='vertical', fontsize="10")
  plt.legend();
  plt.title("Class Distribution");
  plt.xlabel('Classes');
  plt.ylabel('Counts');
  plt.show()

In [ ]:
visualize(y_elec_comb_tr, y_preds_neg_tr)

In [ ]:
#Test and prediction Accuracies on Canadian Election dataset
y_preds_neg_test = model_election_neg.predict(X_elec_comb_test_tf)  ## Using this for further prediction

acc_neg_test = metrics.accuracy_score(y_elec_comb_test, y_preds_neg_test)
report_neg_test = classification_report(y_elec_comb_test,y_preds_neg_test)

#Print Accuracies for comparison
print("The accuracy score for the Test negative tweets is: {0:.3}".format(acc_neg_test))
print("\nThe classification report for the Test negative tweets is :\n {}".format(report_neg_test))
visualize(y_elec_comb_test, y_preds_neg_test)

## **4.a Reason for Poor Performance**

One of the reasons for the model's poor performance is the dataset's severe class imbalance. Over predictions can be seen in the classes with the highest representation: Class 0 and Class 5. Because of the small number of datapoints, the model is unable to generalise well to new data.

## **4.b Suggestion for improvement of model performance**

One method to improve the performance of the model is to generate synthetic datapoints which can improve model generalization. 

Some methods to develop synthetic datapoints are:
1. OverSampling 
2. UnderSampling
3. SMOTE

```
#Examples:
from imblearn.over_sampling import SMOTE
```


##Conclusion

All the learning objectives set at the start of the assignment is applied and met.

-The application of NLP concepts is applied in this assignment.

-Various methods for pasring and cleaning the data is implemented.

-Several text categorization models were investigated, and hyperparameter tweaking was applied to machine learning algorithms to the task of text classificationtraining and testing machine learning algorithms (logistic regression, k-NN, decision trees, random forest, XGBoost, etc.).

-Overall, this project featured a variety of concepts to better understand Sentiment Analysis implementation, develop debates around the observed results, and even provide techniques to improve the implementation.

- NLP either through tweeter or any social media platform gives a great idea to analyse the sentiment of large demography across any given region, Thus helps to predict possible correct consequences of any upcoming event like elections.